In [84]:
import os
import cv2
import datetime
import mediapipe as mp

from glob import glob
from matplotlib import pyplot as plt

In [88]:
mp_face_detection = mp.solutions.face_detection

def crop_face(image, mp_bouding_box):
    if mp_bouding_box is None:
        return image
    
    height, width, _ = image.shape
    
    # bouding box contains negative number in some cases
    x = abs(int(mp_bouding_box.xmin * width))
    y = abs(int(mp_bouding_box.ymin * height))
    w = abs(int(mp_bouding_box.width * width) + x)
    h = abs(int(mp_bouding_box.height * height) + y)

    face_img = image[y:h, x:w]
    
    return face_img


def detect_and_crop_face(image):
    faces = []
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)
        
        if results.detections:        
            for detection in results.detections:
                rect = detection.location_data.relative_bounding_box
                face_img = crop_face(image, rect)
                faces.append(face_img)
        
    return faces

        
def save_face(file_path: str, output_dir: str, file_type='jpg'):
    # mediapip produce more accuracy result
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
        for idx, file_path in enumerate(glob(f'{file_path}/*')):
            name = str(idx)
            image = cv2.imread(file_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_detection.process(image)
            
            if not results.detections:
                continue
            
            for detection in results.detections:
                rect = detection.location_data.relative_bounding_box
                face_img = crop_face(image, rect)
                
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                
                
                cv2.imwrite(output_dir + name + '.jpg', cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB))
                
    
                

In [89]:
save_face('./dataset/train/johnwick/', './crop/johnwick/')
save_face('./dataset/train/ironman/', './crop/iroman/')

